In [1]:
import time
import cv2
import pickle as pkl
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython import display as dp
from collections import deque
import matplotlib.animation as animation

In [2]:
def drawRectangleOnImage(img, bbox, track=0, line=[], colour=(0, 255, 0)):
    b, g, r = colour
    img = cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (int(b), int(g), int(r)), 3)
    img = cv2.putText(img, str(track), (bbox[0], bbox[1] - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (int(b), int(g), int(r)), 3)
    for i in range(1, len(line)):
        img = cv2.line(img, line[i - 1], line[i], (int(b), int(g), int(r)), 3)
    return img

In [3]:
def get_box_center(bbox):
    center = (int(bbox[0]) + ((int(bbox[2]) - int(bbox[0])) // 2)), (int(bbox[1]) + ((int(bbox[3]) - int(bbox[1])) // 2)) # X + Width / 2, Y + Height / 2
    return center

In [4]:
# Detection Pre-Processing 

# Remove overlaps in the same frame
def remove_overlaps(detections_pd, tolerance=0.9):
    row_to_remove = []
    for detection in detections_pd.get('detection'):
        length, _ =detections_pd.shape
        for i in range(length):
            IoU = detection.IoU(detections_pd.iloc[i]['detection'])
            if IoU > tolerance and IoU < 1:
                row_to_remove.append(i)
                
    row_to_remove = np.unique(np.array(row_to_remove))
    detections_pd = detections_pd.drop(index=row_to_remove)
        
    return detections_pd

In [5]:
# Detection to DataFrame
def get_detection_dataframe(detections):
    bboxes = []
    bsizes = []
    lines = deque(maxlen=32)
    bdetections = []
    tracks = list(range(0, len(detections)))
    colours = []
    for i in range(len(detections)):
        colours.append(tuple(np.random.choice(range(256), size=3).astype('int')))
        
    updated = [False]*len(detections)
    
    for detection in detections:
        bbox = np.array(detection.getBBox()).astype('int')
        bboxes.append(bbox)
        
        centers = []
        centers.append(get_box_center(bbox))
        lines.append(centers)

        bsize = int(detection.areaOfRec())
        bsizes.append(bsize)

        bdetections.append(detection)

    detec = {
        'track': tracks,
        'detection': bdetections,
        'bbox': bboxes,
        'size': bsizes,
        'line': lines,
        'colour': colours,
        'updated': updated
    }
    detections_pd = pd.DataFrame(detec)
    detections_pd = detections_pd.sort_values(by=['size'], ascending=False)
    detections_pd = detections_pd.reset_index(drop=True)
    
    return detections_pd

In [6]:
def get_new_track_id(array):
    for cursor in range(30):
        if array[cursor] != cursor:
            return cursor
    return N

In [7]:
def update_track(detections_pd, next_detections_pd, tolerance=0.5):
    detections_pd['updated'] = False
    detections_pd = detections_pd.reset_index(drop=True)
    for index, next_detection in next_detections_pd.iterrows():
        length, _ = detections_pd.shape
        
        # Find overlaps and update if found
        for i in range(length):
            IoU = next_detection['detection'].IoU(detections_pd.iloc[i]['detection'])
            if IoU > tolerance and detections_pd['updated'].iloc[i] != True:
                detections_pd.at[i,'detection'] = next_detection['detection']
                detections_pd.at[i,'bbox'] = next_detection['bbox']
                detections_pd.at[i,'size'] = next_detection['size']
                detections_pd.at[i,'line'].append(next_detection['line'][0])
                detections_pd.at[i,'updated'] = True
                next_detections_pd.at[index, 'updated'] = True
                break
    
    # Drop detections no longer exist
    detections_pd = detections_pd[detections_pd['updated'] == True]
                
    # Start tracking new detections
    if any(next_detections_pd['updated'] == False):
        new_pd = next_detections_pd[next_detections_pd['updated']==False]
        new_pd = new_pd.reset_index(drop=True)
        
        #New Track Number for detection
        
#         max_tracking_id = detections_pd['track'].max()
    
#         for i in range(len(new_pd)):
#             new_pd.at[i, 'track'] = max_tracking_id + 1 + i
        
        
        counter = 0
        for i in range(len(new_pd)):
            while counter in detections_pd['track']:
                counter = counter + 1
            new_pd.at[i, 'track'] = counter
            counter = counter + 2

#         track_list = detections_pd['track'].tolist()
#         track_list = track_list.sort()
#         for index, new in new_pd.iterrows():
#             new_track_id = get_new_track_id(track_list)
#             new_pd.at[index, 'track'] = new_track_id
#             track_list.append(new_track_id)

        detections_pd = pd.concat([detections_pd, new_pd])
    detections_pd = detections_pd.reset_index(drop=True)
    return detections_pd

In [8]:
# Load paths
# detection_path = 'detection_pkls/retinanet_101_detections.pkl'
detection_path = 'detection_pkls/maskRCNN_101_detections.pkl'
data_path = '../datasets/AICity_data/train/S03/c010/'

# Load detections
with open(detection_path , 'rb') as f:
    all_detections = pkl.load(f)

In [9]:
# Pre-process the first frame if needed
# detections_pd = remove_overlaps(detections_pd, 0.5)
# detections_pd

In [10]:
display = False
gif = False

if gif:
    fig, ax = plt.subplots()
    plt.axis('off')

ims = []
detection_history = []

# Get the First Frame
detections_pd = get_detection_dataframe(all_detections['0'])
detection_history.append(detections_pd)

# Load video
vidcap = cv2.VideoCapture(data_path + 'vdo.avi')
_, image = vidcap.read()
num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

# Set plot
if display:
    plt.ion() 
    plt.figure(figsize=(20, 12))
    plt.axis('off') 

# Iterate Frames
# skip = 4
# for frame in tqdm(range(1,num_frames//skip)):
#     for i in range(skip):
#         _, image = vidcap.read()
#     frame = frame*skip

for frame in tqdm(range(1,num_frames-1)):
    _, image = vidcap.read()
    
    next_detections_pd = get_detection_dataframe(all_detections[str(frame)])
    detections_pd = update_track(detections_pd, next_detections_pd, tolerance=0.5)
    detection_history.append(detections_pd)
    
    for index, row in detections_pd.iterrows():
        image = drawRectangleOnImage(image, row['bbox'], track=row['track'], line=row['line'],colour=row['colour'])
        
    if display:
        plt.title('Frame:'+str(frame))
        plt.imshow(image)  
    
    if gif:
        im = ax.imshow(image, animated=True)
        ims.append([im])
        
    if display:
        dp.clear_output(wait=True)
        dp.display(plt.gcf())
        time.sleep(0.000001)
        plt.cla()

if gif:
    ani = animation.ArtistAnimation(fig, ims, interval=10, blit=True, repeat_delay=10000)
    ani.save('Tracking_with_IoU_overlap' + ".gif", writer=animation.PillowWriter(fps=24))
    
# Open a file and use dump()
with open('tracking_history/tracking_history.pkl', 'wb') as file:
    # A new file will be created
    pkl.dump(detection_history, file)

100%|██████████| 534/534 [00:13<00:00, 38.17it/s]


In [11]:
with open('tracking_history/tracking_history.pkl', 'rb') as file:
    # Call load method to deserialze
    tracking_hist = pkl.load(file)

In [12]:
tracking_hist[294]

,track,detection,bbox,size,line,colour,updated
0,0,"Frame 1176, TL [1287.918701171875,361.84143066...","[1287, 361, 1521, 544]",42737,"[(1403, 451), (1404, 452), (1405, 452), (1404,...","(122, 117, 179)",False
1,4,"Frame 1176, TL [1179.367919921875,101.61367034...","[1179, 101, 1230, 163]",3125,"[(1203, 130), (1203, 129), (1203, 130), (1203,...","(189, 74, 108)",False
2,6,"Frame 1176, TL [930.0955200195312,77.887359619...","[930, 77, 1016, 146]",5994,"[(979, 111), (975, 111), (975, 112), (973, 112...","(187, 221, 105)",False
3,8,"Frame 1176, TL [585.4392700195312,74.798614501...","[585, 74, 656, 109]",2507,"[(620, 93), (621, 93), (620, 93), (621, 93), (...","(212, 98, 49)",False
4,7,"Frame 1176, TL [884.7364501953125,94.285987854...","[884, 94, 942, 141]",2748,"[(932, 111), (932, 108), (933, 114), (926, 115...","(70, 73, 75)",False
5,6,"Frame 1176, TL [623.3735961914062,268.06359863...","[623, 268, 817, 413]",28235,"[(685, 127), (688, 123), (704, 122), (716, 120...","(245, 81, 70)",False
6,7,"Frame 1176, TL [563.748046875,97.4318008422851...","[563, 97, 658, 170]",6911,"[(610, 134), (611, 134), (611, 134), (610, 134...","(213, 72, 124)",False
7,1,"Frame 1176, TL [934.5767211914062,340.35903930...","[934, 340, 1187, 556]",54720,"[(1146, 992), (1146, 937), (1134, 891), (1125,...","(177, 69, 186)",False


In [13]:
tracking_hist[295]

,track,detection,bbox,size,line,colour,updated
0,0,"Frame 1180, TL [1288.373291015625,359.71917724...","[1288, 359, 1521, 545]",43437,"[(1403, 451), (1404, 452), (1405, 452), (1404,...","(122, 117, 179)",False
1,4,"Frame 1180, TL [1179.484130859375,101.96245574...","[1179, 101, 1229, 162]",3055,"[(1203, 130), (1203, 129), (1203, 130), (1203,...","(189, 74, 108)",False
2,6,"Frame 1180, TL [925.0783081054688,78.005172729...","[925, 78, 1016, 147]",6353,"[(979, 111), (975, 111), (975, 112), (973, 112...","(187, 221, 105)",False
3,8,"Frame 1180, TL [585.6444091796875,74.968246459...","[585, 74, 657, 111]",2604,"[(620, 93), (621, 93), (620, 93), (621, 93), (...","(212, 98, 49)",False
4,7,"Frame 1180, TL [882.6528930664062,92.579963684...","[882, 92, 942, 141]",2921,"[(932, 111), (932, 108), (933, 114), (926, 115...","(70, 73, 75)",False
5,6,"Frame 1180, TL [612.5098876953125,293.26354980...","[612, 293, 818, 454]",33180,"[(685, 127), (688, 123), (704, 122), (716, 120...","(245, 81, 70)",False
6,7,"Frame 1180, TL [561.8335571289062,99.949554443...","[561, 99, 659, 168]",6674,"[(610, 134), (611, 134), (611, 134), (610, 134...","(213, 72, 124)",False
7,1,"Frame 1180, TL [934.5867309570312,311.03530883...","[934, 311, 1179, 516]",50265,"[(1146, 992), (1146, 937), (1134, 891), (1125,...","(177, 69, 186)",False


In [14]:
# for index, row in detections_pd.iterrows():
#     print(row['colour'])
#     image = drawRectangleOnImage(image, row['bbox'], track=row['track'],colour=row['colour'])

In [15]:
# plt.figure(figsize=(20, 12))
# plt.imshow(image)

In [16]:
# #Test......
# test_num = 60
# detections_pd = get_detection_dataframe(all_detections[str(test_num)])
# next_detections_pd = get_detection_dataframe(all_detections[str(test_num+1)])

In [17]:
# detections_pd

In [18]:
# next_detections_pd

In [19]:
# detections_pd = update_track(detections_pd, next_detections_pd, tolerance=0.5)
# next_detections_pd